In [ ]:
from fsbi.density_estimator import MakePosterior
from fsbi.analyse import ComputeMetrics, default_x, condition
from fsbi.utils import get_density_estim_data, resample_old_data
import matplotlib.pyplot as plt
import torch
import numpy as np
import h5py
import argparse
import yaml
import pickle
from time import time

#### 1/ Polynomial search space

In [ ]:
save_dir = "data_synapsesbi/bg_IF_EEEIIEII_6pPol/"
runs_path = "runs_synapsesbi/bg_IF_EEEIIEII_6pPol/"
metrics = ["rate"] #which metrics is the posterior on
round_name = "pi3_r5to10Hz"


with open(runs_path + "posterior_" + round_name + ".pkl", "rb") as file:
    mk_post = pickle.load(file)
lower_lim=torch.tensor([0.01, 0.01, -2., -2., -2., -2.,
            0.01, 0.01, -2., -2., -2., -2.,
            0.01, 0.01, -2., -2., -2., -2.,
            0.01, 0.01, -2., -2., -2., -2.])
upper_lim=torch.tensor([.1, .1, 2., 2., 2., 2.,
        .1, .1, 2., 2., 2., 2.,
        .1, .1, 2., 2., 2., 2.,
        .1, .1, 2., 2., 2., 2.])

#### 1 bis/ MLP search space

In [ ]:
metrics = ["rate"] #which metrics is the posterior on
round_name = "pi3_r5to10Hz"
save_dir = "../data_synapsesbi/bg_CVAIF_EEIE_T4wvceciMLP/"
runs_path = "../runs_synapsesbi/bg_CVAIF_EEIE_T4wvceciMLP/"
with open(runs_path + "posterior_" + round_name + ".pkl", "rb") as file:
    mk_post = pickle.load(file)
    
lower_lim=torch.tensor([0., 0., -1., -1., -1., -1., #etaEE, etaIE, WpreEE, WpostEE, WpreIE, WpostIE
            -1., -1., -1., -1., -1., -1.,
            -1., -1., -1., -1., -1., -1.,
            -1., -1., -1., -1.])
upper_lim=torch.tensor([1., 1., 1., 1., 1., 1., #etaEE, etaIE, WpreEE, WpostEE, WpreIE, WpostIE
            1., 1., 1., 1., 1., 1.,
            1., 1., 1., 1., 1., 1.,
            1., 1., 1., 1.])

#### 2/ Sample posterior

In [ ]:
#How many samples to generate
post_num_samples = 10

#What should their metric values be (here the only metric is the rate of excitation, and we want it to be between 5 and 10)
default_x = torch.tensor([[np.random.random()*5+5] for i in range(post_num_samples)])

In [ ]:
#The bounds on the plasticity parameters
bounds = {'low':lower_lim,
          'high':upper_lim}

In [ ]:
#The sampling per say
samples = mk_post.rsample(
        default_x,
        bounds)

In [ ]:
#Check that the samples are within the acceptable range
in_prior = [ torch.all(torch.logical_and(lower_lim < samples[i,:], samples[i,:] < upper_lim)) for i in range(len(samples)) ]
print(np.sum(in_prior)/len(in_prior)*100, "% of samples are inside the original prior")

In [ ]:
#Add the nuisance parameter: here only the input rate
concat_tens = torch.cat( (samples, torch.rand( len(samples),1 )*10+5), dim=1) #between 5 and 15Hz

In [ ]:
## save the posterior samples
np.savez(save_dir + "post_samples_" + round_name + ".npz", thetas=concat_tens, default_x=default_x)

Great, you now have new rules to test!

Go to Simulate_samples to simulate these rules with Auryn